In [ ]:
# Step 2: Set up Kaggle API


In [ ]:
# Step 4: Unzip only the img_align_celeba folder (it takes time, big dataset!)



In [ ]:
class CelebADataset:
    def __init__(self, folder_path, limit=80000, img_size=128):
        self.img_size = img_size
        self.folder_path = folder_path
        self.file_names = os.listdir(folder_path)[:limit]  # Limit the number of images
        
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, idx):
        img_name = self.file_names[idx]
        img_path = os.path.join(self.folder_path, img_name)
        
        # Load grayscale image
        X = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        X = cv2.resize(X, (self.img_size, self.img_size))
        X = X.astype('float32') / 255.0
        X = X.reshape(1, self.img_size, self.img_size)  # added channel dim
        
        # Load color image
        y = cv2.imread(img_path)[:, :, ::-1]  # BGR to RGB
        y = cv2.resize(y, (self.img_size, self.img_size))
        y = y.astype('float32') / 255.0
        y = y.transpose(2, 0, 1)  # Change to channel-first format
        
        return X, y
folder_path = r"/content/celeba_data/img_align_celeba/img_align_celeba"
dataset = CelebADataset(folder_path=folder_path, limit=1000, img_size=128)

In [ ]:
# Example: Get first sample
X_sample, y_sample = dataset[0]
print(f"Grayscale image shape: {X_sample.shape}")
print(f"Color image shape: {y_sample.shape}")

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
a, b = next(iter(train_loader))
a.shape, b.shape

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.Encoder = nn.Sequential(

            nn.Conv2d(1, 64, (3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, (3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, (3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, (3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 1024, (3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(1024),
            nn.MaxPool2d(kernel_size=2, stride=2),

        )

        self.Decoder = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, (3,3), stride=2, padding=1, output_padding=1),
            nn.ReLU(),

            nn.ConvTranspose2d(512, 256, (3,3), stride=2, padding=1, output_padding=1),
            nn.ReLU(),

            nn.ConvTranspose2d(256, 128, (3,3), stride=2, padding=1, output_padding=1),
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, (3,3), stride=2, padding=1, output_padding=1),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 3, (3,3), stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x


In [ ]:
# Create model and move to GPU
model = Autoencoder().to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 50
train_losses = []

for epoch in range(epochs):
    model.train()
    total_epoch_loss = 0
    
    for gray, col in tqdm(train_loader):
        # Move data to GPU
        gray = gray.to(device)
        col = col.to(device)
        
        # Forward pass
        outputs = model(gray)
        loss = criterion(outputs, col)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_epoch_loss += loss.item()
    
    # Print epoch results
    avg_loss = total_epoch_loss / len(train_loader)
    print(f"EPOCH: {epoch+1}, LOSS: {avg_loss}")
    train_losses.append(avg_loss)

In [ ]:
plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Get a batch of data
model.eval()
with torch.no_grad():
    gray, col = next(iter(train_loader))
    gray = gray.to(device)
    
    # Generate predictions
    outputs = model(gray)
    
    # Move back to CPU for visualization
    gray = gray.cpu()
    outputs = outputs.cpu()

In [ ]:
plt.figure(figsize=(15, 6))
num_samples = 5

for i in range(num_samples):
    # Plot input (grayscale)
    plt.subplot(2, num_samples, i + 1)
    plt.imshow(gray[i].squeeze(), cmap='gray')
    plt.title('Input')
    plt.axis('off')
    
    # Plot output
    plt.subplot(2, num_samples, i + 1 + num_samples)
    if outputs.shape[1] == 1:  # Grayscale output
        plt.imshow(outputs[i].squeeze(), cmap='gray')
    else:  # Color output
        plt.imshow(outputs[i].permute(1, 2, 0).squeeze())
    plt.title('Output')
    plt.axis('off')

plt.show()